Old one

In [ ]:
class Task:

    def __init__(self, job_id, task_id, ta, C, M):
        global mu

        self.jid = job_id
        self.tid = task_id
        self.ta = ta
        self.C = C
        self.M = M
        self.X = C / mu
        self.end_time = 0
        self.remaining_X = 0 
        self.waiting_time = 0

    # To avoid comaprison problems among different Tasks.
    def __lt__(self, other):
        return self.ta < other.ta


class Server:

    def __init__(self, server_id):
        self.server_id = server_id
        self.M = 0
        self.time = 0
        self.workload = 0 
        self.running_tasks = []
        self.queue = queue.PriorityQueue()
        self.n_tasks = 0
        self.n_queue = 0


    def check_server_workload(self, time):
    # Time == t_a of new row
        while self.time < time:

            if self.running_tasks:
                self.running_tasks = sorted(self.running_tasks, key=lambda obj: obj.remaining_X)

                first_ending_task = self.running_tasks[0]

                # Think using every time the task that stop first.
                if first_ending_task.remaining_X < time - self.time:

                    # Updating times.
                    first_ending_task.end_time = first_ending_task.waiting_time + first_ending_task.X
                    self.M -= first_ending_task.M
                    if self.running_tasks:
                        for task in self.running_tasks:
                            task.remaining_X -= first_ending_task.remaining_X
                    self.time += self.running_tasks.pop(0).remaining_X

                    if not self.queue.empty():
                        if self.M + self.queue.queue[0][1].M < 1:
                            self.M += self.queue.queue[0][1].M
                            self.queue.queue[0][1].waiting_time = self.time - self.queue.queue[0][1].ta
                            self.running_tasks.append(self.queue.get()[1])
                            self.running_tasks = sorted(self.running_tasks, key=lambda obj: obj.remaining_X)
                else:
                    for task in self.running_tasks:
                        task.remaining_X -= time - self.time
                    self.time = time
            else:
                self.time = time
                
        # Workload of a server to Dispacher's decision.
        self.workload = 0
        for task in self.queue.queue:
            self.workload += task[1].X

        for task in self.running_tasks:
            self.workload += task.remaining_X


    def assign_task_to_server(self, task):
        
        self.n_tasks += 1
        
        # If the memory is available do not pass for the queue.
        if self.queue.empty() and self.M == 0:
            self.time = task.ta

        if self.queue.empty() and self.M + task.M < 1:
            self.running_tasks.append(task)
            self.M += task.M
        else:
            self.queue.put((task.ta, task)) 
            self.n_queue += 1
        

    def conclude_server_work(self):

        # Finish the work of tasks after running all the rows of dataframe
        while not self.queue.empty(): 
            first_ending_task = self.running_tasks[0]
            first_ending_task.end_time = first_ending_task.waiting_time + first_ending_task.X
            self.M -= first_ending_task.M
            if self.running_tasks:
                for task in self.running_tasks:
                    task.remaining_X -= first_ending_task.remaining_X
            self.time += self.running_tasks.pop(0).remaining_X

            # If next task in queue can be storage in memory.
            if self.M + self.queue.queue[0][1].M < 1:
                self.M += self.queue.queue[0][1].M
                self.queue.queue[0][1].waiting_time = self.time - self.queue.queue[0][1].ta
                self.running_tasks.append(self.queue.get()[1])
                self.running_tasks = sorted(self.running_tasks, key=lambda obj: obj.remaining_X)


        # Close reamining ones still running in the server.
        if self.running_tasks:
            for task in self.running_tasks:
                task.end_time = task.waiting_time + task.X



# Dispatching function.
def LWL(task, servers):
    
    min_workload = float('inf')
    selected_server = None

    for server in servers.values():
        server.check_server_workload(task.ta)
        if server.workload < min_workload:
            min_workload = server.workload
            selected_server = server

    selected_server.assign_task_to_server(task)

new one

In [ ]:
class Server:

    def __init__(self, server_id):
        self.server_id = server_id
        self.M = 0
        self.time = 0
        self.workload = 0 
        self.queue = queue.PriorityQueue()
        self.n_tasks = 0
        self.inactive_time = 0

    def assign_task(self, task):
        self.n_tasks += 1
        self.queue.put((task.ta, task))

    def simulate_work(self, task):
        
        while self.time < task.ta and not self.queue.empty():
            next_task = self.queue.queue[0][1]
            waiting_time = self.time - next_task.ta
            next_task.end_time += waiting_time

            if next_task.X + self.time < task.ta:
                running_task = self.queue.get()[1]
                self.time += running_task.X
            else:
                next_task.X = (next_task.X + self.time) - task.ta
                self.time = task.ta
        
        self.inactive_time += task.ta - self.time
        self.time = task.ta
        self.workload = sum([t[1].X for t in self.queue.queue])

    def finish_queue(self):
        while not self.queue.empty():
            running_task = self.queue.get()[1]
            self.time += running_task.X
            running_task.end_time += self.time - running_task.ta

In [ ]:
# Dispatching function.
def LWL(task, servers):
    
    min_workload = float('inf')
    selected_server = None

    for server in servers.values():
        server.simulate_work(task)
        if server.workload < min_workload:
            min_workload = server.workload
            selected_server = server

    selected_server.assign_task(task)

In [ ]:
# data = data.head(200_000)

# Creating servers objects.
servers = {i: Server(i) for i in range(N)}


# Creating tasks object.
tasks = {}
for index, row in tqdm(data.iterrows()):
    jid, tid, ta, C, M = row.values
    tasks[index] = Task(int(jid), int(tid), int(ta), float(C), float(M))


# Execute dispatcher for each task.
for idx, task in tqdm(tasks.items()):
    LWL(task, servers) 


# Complete the tasks remained in the servers.
for server in tqdm(servers.values()):
    server.finish_queue()